# Imports

In [1]:
# regex
import re

# pandas + numpy
import numpy as np
import pandas as pd

# setting pandas options
pd.set_option('display.max_colwidth', 200)


# storing and loading models
import pickle

# to set types for functions
from typing import Tuple

# Plotting
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


# gpu debug
import torch

# setting device to use GPU for NLP backend if you have GPU available
device = "cuda" if torch.cuda.is_available() else "cpu"


# SBERT
from sentence_transformers import SentenceTransformer

# UMAP
from umap import UMAP

#HDBSCAN
from hdbscan import HDBSCAN

# topic finding
from sklearn.feature_extraction.text import TfidfVectorizer

# Loading model from pickle if possible, to avoid downloading it again
try:
    model = pickle.load(open(f'model-{device}.pkl', 'rb'))

    model_load = True

except:
    model = SentenceTransformer('all-mpnet-base-v2', device=device)
    pickle.dump(model, open(f'model-{device}.pkl', 'wb'))

    model_load = False

print(f"""
GPUs detected:          {torch.cuda.device_count()}
Using GPU:              {torch.cuda.is_available()}
Device:                 {device}
Got model from pickle:  {model_load}
""")

/opt/miniconda3/envs/intro-ai/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



GPUs detected:          0
Using GPU:              False
Device:                 cpu
Got model from pickle:  False



# Functions

In [2]:
def tfidf_most_relevant_word(input: list, num_words=5) -> list:
  """
  Function that finds the most relevant words per cluster id.

  Args:
      input (list): A list of title strings aggregated by cluster id.
      num_words (int, optional): How many words you want. Defaults to 5.

  Returns:
      list: Returns a list of most relevant words, with lenght of unique cluster Ids
  """

  most_relevant_words = []
  
  for corpus in input:
        
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(corpus)
    
    importance = np.argsort(np.asarray(X.sum(axis=0)).ravel())[::-1]
    tfidf_feature_names = np.array(vectorizer.get_feature_names_out()) # get_feature_names
    most_relevant_words.append(tfidf_feature_names[importance[:num_words]])

  return most_relevant_words

## Cleaning

In [3]:
def string_cleaner(input: str) -> str:
    """
    Function to clean up strings.

    Args:
        input (str): String to be cleaned.

    Returns:
        str: Cleaned string.
    """
    
    # turning lowercase
    input = input.lower()

    # removing punctuation and other non-alphanumeric characters
    input = re.sub(r'[^\w\s]', '', input)
    
    return input

## Topic Modeling

In [4]:
def tfidf_most_relevant_word(input: list, num_words=5) -> list:
  """
  Function that finds the most relevant words per cluster id.

  Args:
      input (list): A list of title strings aggregated by cluster id.
      num_words (int, optional): How many words you want. Defaults to 5.

  Returns:
      list: Returns a list of most relevant words, with lenght of unique cluster Ids
  """

  most_relevant_words = []
  
  for corpus in input:
        
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(corpus)
    
    importance = np.argsort(np.asarray(X.sum(axis=0)).ravel())[::-1]
    tfidf_feature_names = np.array(vectorizer.get_feature_names_out()) # get_feature_names
    most_relevant_words.append(tfidf_feature_names[importance[:num_words]])

  return most_relevant_words



def topic_by_clusterId(result: pd.DataFrame) -> dict:
  """
  Function that maps topics to cluster ids.

  Args:
      result (pd.DataFrame): Dataframe with cluster ids and topics.

  Returns:
      dict: Dictionary with cluster ids as keys and topics as values.
  """

  #print(result.isna().sum())

  df_group = result[["titles", "cluster_label"]].groupby("cluster_label").agg(list).reset_index()

  df_group["topics"] = tfidf_most_relevant_word(df_group["titles"])

  return dict(zip(df_group.cluster_label, df_group.topics))

## Plotting Functions

In [5]:
# when you actually cast the type here, then it works with how pandas casts types and you don't have to worry about copying seriers
def result_df_maker(embeddings: np.ndarray, cluster_labels: np.ndarray, titles: np.ndarray) -> pd.DataFrame:
  """
  Function to make a dataframe with the embeddings, cluster labels, topic per cluster label and titles.

  Args:
      embeddings (np.ndarray): 2D array of embeddings.
      cluster_labels (np.ndarray): array of cluster labels.
      titles (np.ndarray): array of titles.

  Returns:
      pd.DataFrame: Dataframe with embeddings, cluster labels, topics per cluster, and titles.
  """
  result = pd.DataFrame(embeddings, columns=['x', 'y'])

  result["titles"] = titles

  result["cluster_label"] = cluster_labels

  topic_dict = topic_by_clusterId(result)

  result["topics"] = result["cluster_label"].apply(lambda x: topic_dict[x])

  result["topics"] = result["topics"].apply(lambda x: " ".join(x))

  return result

def result_splitter(result: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
  """
  Function to split the dataframe into two dataframes, one for clustered and one for outliers.

  Args:
      result (pd.DataFrame): Dataframe with embeddings, cluster labels, topics per cluster, and titles.

  Returns:
      Tuple[np.ndarray, np.ndarray]: Tuple of two dataframes, one for clustered and one for outliers.
  """

  clustered = result.loc[result.cluster_label != -1, :]
  outliers = result.loc[result.cluster_label == -1, :]
  return clustered, outliers

# the cavalry is not here, but it's fine! Why? I am here!
def result_tracer(clustered: pd.DataFrame, outliers: pd.DataFrame) -> Tuple[go.Scattergl, go.Scattergl]:
  """
  Function to make a scatter traces of the clustered and outliers.

  Args:
      clustered (pd.DataFrame): clustered dataframe to be colored by cluster and get hover data
      outliers (pd.DataFrame): outlier data frame with grey color and no hover data

  Returns:
      Tuple[go.Scattergl, go.Scattergl]: Tuple of two scatter traces.
  """

  trace_cluster = go.Scattergl(
    x=clustered.x, 
    y=clustered.y, 
    mode="markers", 
    name="Clustered",

    # styling markers
    marker=dict(
      size=2, 
      color=clustered.cluster_label,
      colorscale="Rainbow"
    ), 

    # setting hover text to the titles of the videos
    hovertemplate="<b>Title:</b> %{customdata[0]} <br><b>Topics:</b> %{customdata[1]} <br><b>Cluster Id:</b> %{customdata[2]}<extra></extra>", 
    customdata=np.column_stack([clustered.titles, clustered.topics, clustered.cluster_label]),
  )

  trace_outlier = go.Scattergl(
    x=outliers.x,
    y=outliers.y,
    mode="markers",
    name="Outliers",

    marker=dict(
      size=1,
      color="grey"
    ),

    hovertemplate="Outlier<extra></extra>"
  )

  return trace_cluster, trace_outlier

def result_tracer_wrapper(uembs: np.ndarray, cluster_labels: np.ndarray, titles: np.ndarray) -> Tuple[go.Scattergl, go.Scattergl]:
  """
  Function to make a scatter traces of the clustered and outliers.

  Args:
      uembs (np.ndarray): 2D array of embeddings.
      cluster_labels (np.ndarray): array of cluster labels.
      titles (np.ndarray): array of titles.

  Returns:
      Tuple[go.Scattergl, go.Scattergl]: Tuple of two scatter traces.
  """

  result = result_df_maker(uembs, cluster_labels, titles)
  clustered, outliers = result_splitter(result)
  trace_cluster, trace_outlier = result_tracer(clustered, outliers)
  return trace_cluster, trace_outlier

In [6]:
def subplotter(trace_nested_list: list, titles: list, base_size=1000) -> go.Figure:
    """
    Function to make a figure with subplots of the clustered and outliers.

    Args:
        trace_nested_list (list): list holding rows of columns, each column holding traces. 
        titles (list): Titles for the subplots
        base_size (int, optional): Base size of the sub plots. Defaults to 1000.

    Returns:
        go.Figure: Figure with subplots.
    """
    
    row_count = len(trace_nested_list)
    col_count = len(trace_nested_list[0])
    
    fig = make_subplots(
        rows=row_count, 
        cols=col_count,
        subplot_titles=(titles),
        vertical_spacing=0.02,
        horizontal_spacing=0.02
    )

    for i, row in enumerate(trace_nested_list):
        for j, col in enumerate(row):

            # adding both outlieers and clustered
            for trace in col:
                fig.add_trace(trace, row=i+1, col=1)
    
    # figure settings
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    
    fig.update_layout(width=base_size*col_count, height=base_size*row_count, plot_bgcolor='rgba(250,250,250,1)')

    return fig

## Saving / Showing Plots

In [7]:
def fig_show_save(fig: go.Figure, filename: str, show=True):
  """
  Function to show and save a figure.

  Args:
      fig (go.Figure): fig to be saved and shown
      filename (str): filename to save the figure, without extension
      show (bool, optional): Option to disable showing of figure (in case too big for notebook). Defaults to True.
  """
  
  # writing both interactible .html and static image .png
  fig.write_html(f"figures/{filename}.html")
  fig.write_image(f"figures/{filename}.png")

  if show: 
    fig.show()

# Data Part

In this part we read the csv-file "USvideos.csv" using pandas "read_csv" function to create a pandas database structure. We are only concentrating on the titles in this assignment so we create the database "df" with only the titles from the df_whole. 
Then there is an exception for computers not having the capability of gpu-acceleration during the ML part. This just takes a smaller version of the database to make it faster to run through. 
The last two lines are just to be able to see what we just created. We see the "shape" of the df and then the first 3 lines.

In [8]:
# got data from kaggle: https://www.kaggle.com/datasets/datasnaek/youtube-new?resource=download

df_whole = pd.read_csv("data/USvideos.csv")

df = df_whole[["title"]].copy()

# if your computer does not have GPU support, you can use a sample of the dataset instead to make it run in a reasonable time
# if you want to use the full dataset even wihtout GPU in case you have a very strong CPU, then you can just comment out the next line
if device == "cpu": df = df.sample(frac=0.05)

print(df.shape)

df.head(3)

(2047, 1)


,title
37887,BLIND GIRL DOES MY MAKEUP ft. MOLLY BURKE
23348,Gotta Catch ‘Em All!
11135,Bloopers 2017 !


## Cleaning

The titles from youtube are cluttered with emojis and special characters that does not give any information and/or create problems for the ML-algorithms that we are going to use later. Therefore we can run the string_cleaner function and remove any characters we don't need in the later parts.
The last line is just to see that the new clean titles actually stuck to the df.

In [9]:
df["title_clean"] = df["title"].apply(string_cleaner)

df.head(3)

,title,title_clean
37887,BLIND GIRL DOES MY MAKEUP ft. MOLLY BURKE,blind girl does my makeup ft molly burke
23348,Gotta Catch ‘Em All!,gotta catch em all
11135,Bloopers 2017 !,bloopers 2017


# ML Part

## Getting Encodings

Embs is a numpy nd-array (n-dimential array) created from the clean titles in the df, in this case a 3-dimential ndarray. The numpy array of the titles are then sent to the pickle model to be processed. Pickle is a library that takes in a python object and creating a "byte-stream" from them. This ends up being a whole bunch of floats in a 3-dimential array.

The next part is printing some values from the embs variable, just to see what we created.

In [10]:
embs = model.encode(df["title_clean"].to_numpy())

In [11]:
print(f"""
{type(embs)}
hvor langt er selve embs: {len(embs)}
hvor langt et er et element: {len(embs[0])}
hva er første element i embs:
{embs[0][0:20]}
""")


<class 'numpy.ndarray'>
hvor langt er selve embs: 2047
hvor langt et er et element: 768
hva er første element i embs:
[-0.05261553  0.01696352 -0.00403966 -0.07176481 -0.01395654  0.00447699
  0.00080223  0.0265346   0.00198676 -0.00415409 -0.01729897 -0.03933381
  0.01172205  0.12617272 -0.03456242 -0.01166943 -0.04256149  0.0173528
 -0.00440879  0.02510993]



Now we put the embs floats in the 3-dimential ndarray into the df database under the name "embs". We have to use "list" to list the items so the values gets separated correctly with the df.

In [12]:
df["embs"] = list(embs)
df.head(3)

,title,title_clean,embs
37887,BLIND GIRL DOES MY MAKEUP ft. MOLLY BURKE,blind girl does my makeup ft molly burke,"[-0.052615527, 0.016963525, -0.0040396596, -0.07176481, -0.013956536, 0.004476993, 0.00080222735, 0.026534596, 0.001986759, -0.0041540926, -0.017298967, -0.03933381, 0.011722054, 0.12617272, -0.03..."
23348,Gotta Catch ‘Em All!,gotta catch em all,"[-0.011540012, 0.08560703, -0.018570684, -0.008629536, 0.0099618705, -0.03198622, -0.057538256, 0.013699182, 0.0050628847, 0.011320668, -0.020062584, -0.016542122, 0.0058959387, 0.019945221, 0.032..."
11135,Bloopers 2017 !,bloopers 2017,"[0.0077608568, 0.013997772, 0.009772727, -0.045653716, -0.019328464, -0.036193117, -0.018980745, 0.04604681, -0.084601805, -0.018199096, 0.011673877, -0.0155382985, -0.015452402, 0.082538694, -0.0..."


## Dimensinality Reduction

Here we are using UMAP algorithm to create 2-dimential array from a 3-dimential one in embs. UMAP sees the correlation between the 3 coordinates and creates a accurate version in 2-dimentions. We are doing this so that we can visualize the final cluster plotting in 2-dimentions. The values n_neighbors and min_dist are values that can be used to fine-tune the algorithm, to make us get the values we want. 
The fit_transform function does the processing itself. umap variable was just a model from the UMAP library.

In [13]:
umap = UMAP(n_neighbors=20, min_dist=0.1)

embs_2d = umap.fit_transform(embs)

In [14]:
print(f"""
{type(embs_2d)}
hvor langt er selve embs_2d: {len(embs_2d)}
hvor langt et er et element: {len(embs_2d[0])}
hva er første element i embs_2d:
{embs_2d[0]}
""")


<class 'numpy.ndarray'>
hvor langt er selve embs_2d: 2047
hvor langt et er et element: 2
hva er første element i embs_2d:
[11.666381 10.037041]



Here we put the new 2-dimential values into the df, aswell as printing it to see that it worked.

In [15]:
df["embs_2b"] = list(embs_2d)

df.head(3)

,title,title_clean,embs,embs_2b
37887,BLIND GIRL DOES MY MAKEUP ft. MOLLY BURKE,blind girl does my makeup ft molly burke,"[-0.052615527, 0.016963525, -0.0040396596, -0.07176481, -0.013956536, 0.004476993, 0.00080222735, 0.026534596, 0.001986759, -0.0041540926, -0.017298967, -0.03933381, 0.011722054, 0.12617272, -0.03...","[11.666381, 10.037041]"
23348,Gotta Catch ‘Em All!,gotta catch em all,"[-0.011540012, 0.08560703, -0.018570684, -0.008629536, 0.0099618705, -0.03198622, -0.057538256, 0.013699182, 0.0050628847, 0.011320668, -0.020062584, -0.016542122, 0.0058959387, 0.019945221, 0.032...","[7.139455, 8.178256]"
11135,Bloopers 2017 !,bloopers 2017,"[0.0077608568, 0.013997772, 0.009772727, -0.045653716, -0.019328464, -0.036193117, -0.018980745, 0.04604681, -0.084601805, -0.018199096, 0.011673877, -0.0155382985, -0.015452402, 0.082538694, -0.0...","[8.662644, 6.0648923]"


## Clustering 2D data

This is the plotting of the values we got from umap. We are setting it up so the x-values are the first number before the comma and y-values from the second value after the comma. Then the figure is updated with values of the size of the window and some other visual values. 

Lastly the fig_show_save function is run. This function creates the plot itself and also saves a html-file and png file of the same model.

In [16]:
fig = px.scatter(x=embs_2d[:,0], y=embs_2d[:,1])

fig.update_layout(width=800, height=800)
fig.update_traces(marker=dict(size=2))

fig_show_save(fig, "test")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Now we have to cluster the plots into categories. We do this by using the HDBSCAN algorithm. We create clusters by running the HDBSCAN and giving the paramters we like. These can be changed to fine-tune results. We give HDBSCAN embs_2d values and run the function. This gives every title an ID of which cluster it is in. If the function finds the title as an outlier, it gives them the ID of -1. 

We lastly print out the number of clusters and how many outliers it found. 

In [17]:
clusters_2d = HDBSCAN(min_cluster_size=10, cluster_selection_method="leaf").fit(embs_2d)

print(f"""
    Number of clusters: {len(set(clusters_2d.labels_)) - 1}
    Number of rows as outliers: {clusters_2d.labels_.tolist().count(-1)}
""")


    Number of clusters: 43
    Number of rows as outliers: 1153



We can now put the new cluster-label-ID's into the df.

In [18]:
df["cluster_id"] = clusters_2d.labels_
df.head(3)


,title,title_clean,embs,embs_2b,cluster_id
37887,BLIND GIRL DOES MY MAKEUP ft. MOLLY BURKE,blind girl does my makeup ft molly burke,"[-0.052615527, 0.016963525, -0.0040396596, -0.07176481, -0.013956536, 0.004476993, 0.00080222735, 0.026534596, 0.001986759, -0.0041540926, -0.017298967, -0.03933381, 0.011722054, 0.12617272, -0.03...","[11.666381, 10.037041]",42
23348,Gotta Catch ‘Em All!,gotta catch em all,"[-0.011540012, 0.08560703, -0.018570684, -0.008629536, 0.0099618705, -0.03198622, -0.057538256, 0.013699182, 0.0050628847, 0.011320668, -0.020062584, -0.016542122, 0.0058959387, 0.019945221, 0.032...","[7.139455, 8.178256]",34
11135,Bloopers 2017 !,bloopers 2017,"[0.0077608568, 0.013997772, 0.009772727, -0.045653716, -0.019328464, -0.036193117, -0.018980745, 0.04604681, -0.084601805, -0.018199096, 0.011673877, -0.0155382985, -0.015452402, 0.082538694, -0.0...","[8.662644, 6.0648923]",-1


# Results

## Plotting the results

We can now plot our results. We use the function result_tracer_wrapper to create colors for each cluster and putting them on their representing dot. The figure is created and shown using the fig_show_save function.

In [19]:
trace_cluster_2d, trace_outlier_2d = result_tracer_wrapper(embs_2d, clusters_2d.labels_, df["title_clean"].to_numpy())


col11 = [trace_cluster_2d, trace_outlier_2d]


row1 = [col11]


trace_list = [row1]

fig = subplotter(trace_list, ["Topics by HDBSCAN Cluster", ])

fig_show_save(fig, "topics-by-hdbscan-clusters")

## Showing topic per cluster

Lastly to show the names of every title per cluster we group then by the groupby function and them putting it in a list and lastly adding it to the dfcg database and showing the 3 first columns. Now we see every colmn by each cluster id. 

In [20]:
dfcg = df.groupby(["cluster_id"])
dfcg = dfcg.agg(list)
dfcg = dfcg.reset_index()
dfcg.head(3)

,cluster_id,title,title_clean,embs,embs_2b
0,-1,"[Bloopers 2017 !, How do I stop these tears (YIAY #411), Mario Batali taken off The Chew amid sexual harassment claims, How to Add Headphone Audio Port to Google Home Mini, Eating BRUNCH at Taiwan...","[bloopers 2017 , how do i stop these tears yiay 411, mario batali taken off the chew amid sexual harassment claims, how to add headphone audio port to google home mini, eating brunch at taiwan 7el...","[[0.0077608568, 0.013997772, 0.009772727, -0.045653716, -0.019328464, -0.036193117, -0.018980745, 0.04604681, -0.084601805, -0.018199096, 0.011673877, -0.0155382985, -0.015452402, 0.082538694, -0....","[[8.662644, 6.0648923], [8.863833, 9.41119], [10.558565, 7.305476], [5.28122, 7.0902843], [5.976647, 11.540803], [6.64195, 8.883712], [8.726426, 8.199493], [8.072727, 8.5096655], [8.256806, 9.6531..."
1,0,"[Liam Payne Goes Sneaker Shopping With Complex, YoungBoy Never Broke Again Goes Sneaker Shopping With Complex, Michael B. Jordan Goes Sneaker Shopping With Complex, How to Control Your Boyfriend |...","[liam payne goes sneaker shopping with complex, youngboy never broke again goes sneaker shopping with complex, michael b jordan goes sneaker shopping with complex, how to control your boyfriend h...","[[-0.012741206, 0.018409621, -0.021878442, -0.015849615, 0.022089882, 0.017322535, 0.04455682, -0.011976722, 0.010065375, 0.046709772, 0.0013516964, 0.034348123, 0.005432029, 0.031386606, -0.00791...","[[7.8715067, 11.529738], [7.8825097, 11.531931], [7.852327, 11.535404], [7.9078655, 11.242978], [7.923539, 11.478437], [7.869447, 11.530656], [7.8711677, 11.532125], [7.869071, 11.533429], [7.8859..."
2,1,"[Men arrested at Starbucks speak out, Vermilion Parish teacher gets arrested at Vermilion Parish school board meeting, Trump meets with students following Florida shooting, Broward County Authori...","[men arrested at starbucks speak out, vermilion parish teacher gets arrested at vermilion parish school board meeting, trump meets with students following florida shooting, broward county authori...","[[0.012830332, 0.0074575534, -0.017840968, 0.0014738155, -0.015911268, 0.029376786, 0.07168556, -0.059605915, -0.042199347, 0.0008557752, -0.012026298, -0.010673283, 0.014376149, -0.011085361, -0....","[[11.826118, 7.178493], [11.860192, 7.2147346], [11.740818, 7.274625], [11.81458, 7.273465], [11.788018, 7.1989655], [11.8184, 7.2830725], [11.731624, 7.269337], [11.631336, 7.170693], [11.695262,..."
